In [5]:
sp500 = 'https://www.blackrock.com/lu/individual/products/253743/ishares-sp-500-b-ucits-etf-acc-fund/1509774406454.ajax?fileType=csv&fileName=CSPX_holdings&dataType=fund'

In [6]:
import pandas as pd
CSPX_fund_breakdown = pd.read_csv(sp500,skiprows=2)
CSPX_fund_breakdown

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,Price,Location,Exchange,Market Currency
0,AAPL,APPLE INC,Information Technology,Equity,"7,754,335,228.12",6.73,"7,754,335,228.12","34,065,524.00",227.63,United States,NASDAQ,USD
1,NVDA,NVIDIA CORP,Information Technology,Equity,"7,177,738,274.40",6.23,"7,177,738,274.40","55,281,410.00",129.84,United States,NASDAQ,USD
2,MSFT,MICROSOFT CORP,Information Technology,Equity,"6,865,524,740.25",5.96,"6,865,524,740.25","16,755,399.00",409.75,United States,NASDAQ,USD
3,AMZN,AMAZON COM INC,Consumer Discretionary,Equity,"4,832,819,559.15",4.20,"4,832,819,559.15","21,090,201.00",229.15,United States,NASDAQ,USD
4,META,META PLATFORMS INC CLASS A,Communication,Equity,"3,510,385,314.56",3.05,"3,510,385,314.56","4,912,928.00",714.52,United States,NASDAQ,USD
...,...,...,...,...,...,...,...,...,...,...,...,...
505,MXN,MXN CASH,Cash and/or Derivatives,Cash,"413,767.47",0.00,"413,767.47","8,511,300.00",4.86,Mexico,-,MXN
506,GBP,GBP CASH,Cash and/or Derivatives,Cash,"310,579.42",0.00,"310,579.42","250,104.00",124.18,United Kingdom,-,GBP
507,EUR,EUR CASH,Cash and/or Derivatives,Cash,"1,783.62",0.00,"1,783.62","1,724.00",103.44,European Union,-,EUR
508,ESH5,S&P500 EMINI MAR 25,Cash and/or Derivatives,Futures,0.00,0.00,"158,496,900.00",524.00,"6,049.50",-,Index And Options Market,USD


In [7]:
CSPX_holdings = CSPX_fund_breakdown[CSPX_fund_breakdown['Asset Class'] == 'Equity'][['Ticker','Name','Weight (%)']]
#sort by weight desc, take top 100
CSPX_top_100 = CSPX_holdings.sort_values('Weight (%)',ascending=False).head(100)

In [8]:
CSPX_top_100

,Ticker,Name,Weight (%)
0,AAPL,APPLE INC,6.73
1,NVDA,NVIDIA CORP,6.23
2,MSFT,MICROSOFT CORP,5.96
3,AMZN,AMAZON COM INC,4.20
4,META,META PLATFORMS INC CLASS A,3.05
...,...,...,...
95,ADI,ANALOG DEVICES INC,0.20
96,CB,CHUBB LTD,0.20
97,KKR,KKR AND CO INC,0.20
98,KLAC,KLA CORP,0.20


In [4]:
import pandas as pd
import feedparser
from transformers import pipeline
import time

# Initialize a sentiment analysis pipeline (Hugging Face model)
pipe = pipeline(task= 'text-classification', model = "ProsusAI/finbert")

def generate_ticker_sentiment_df(ticker):
    res_url = f'https://finance.yahoo.com/rss/headline?s={ticker}'
    feed = feedparser.parse(res_url)
    keyword = ticker.lower()
    
    positive_count = 0
    neutral_count = 0
    negative_count = 0
    total_score = 0
    num_articles = 0

    for entry in feed.entries:
        if keyword not in entry.summary.lower():
            continue
        
        try:
            sentiment = pipe(entry.summary)[0]
        except Exception as e:
            print(f"Error analyzing sentiment for {ticker}: {e}")
            continue

        label = sentiment["label"].lower()
        score = sentiment["score"]

        if label == "positive":
            positive_count += 1
            total_score += score
        elif label == "negative":
            negative_count += 1
            total_score -= score
        elif label == "neutral":
            neutral_count += 1

        num_articles += 1

    # Adjust sentiment score to account for neutral sentiment
    final_score = total_score / num_articles if num_articles > 0 else 0

    return {
        "Ticker": ticker,
        "Positive Headlines": positive_count,
        "Neutral Headlines": neutral_count,
        "Negative Headlines": negative_count,
        "Total Articles": num_articles,
        "Sentiment Score": final_score
    }

def generate_cspx_sentiment_df(CSPX_top_100):
    sentiment_data = []

    for i, ticker in enumerate(CSPX_top_100["Ticker"]):
        print(f"Processing {ticker} ({i + 1}/{len(CSPX_top_100)})...")
        ticker_sentiment = generate_ticker_sentiment_df(ticker)
        sentiment_data.append(ticker_sentiment)

    # Create a DataFrame with sentiment data
    sentiment_df = pd.DataFrame(sentiment_data)

    # Merge the sentiment data back with the original CSPX_top_100 DataFrame
    result_df = CSPX_top_100.merge(sentiment_df, on="Ticker")
    return result_df

# Example usage
CSPX_sentiment_df = generate_cspx_sentiment_df(CSPX_top_100)
print(CSPX_sentiment_df)


from datetime import datetime
datetime = str(datetime.today()).split()[0]
CSPX_sentiment_df.to_csv(f"CSPX_sentiment_analysis_{datetime}.csv", index=False)

NameError: name 'CSPX_top_100' is not defined